# Glass Expert Optimization Example

This notebook demonstrates how to use the `GlassExpert` optimizer in Optiland to optimize an optical system with both continuous (e.g., radii, thicknesses) and categorical (glass types) variables.

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import optiland.backend as be
from optiland import optic, optimization
from optiland.materials import glasses_selection
from optiland.visualization import plot_optic_history_matplotlib, plot_error_history_matplotlib

## 1. Define the Optical System

We'll use a simple Cooke Triplet as the starting point for our optimization.

In [ ]:
class CookeTripletStartPoint(optic.Optic):
    def __init__(self):
        super().__init__()
        self.add_surface(index=0, radius=be.inf, thickness=be.inf) # Object surface
        self.add_surface(index=1, radius=25.0, thickness=2.5, material="N-BK7")
        self.add_surface(index=2, radius=-150.0, thickness=7.5)
        self.add_surface(index=3, radius=-25.0, thickness=1.5, material="N-F2") # Stop surface
        self.add_surface(index=4, radius=25.0, thickness=5.0)
        self.add_surface(index=5, radius=150.0, thickness=2.5, material="N-BK7")
        self.add_surface(index=6, radius=-25.0, thickness=40.0)
        self.add_surface(index=7) # Image surface

        self.surfaces[3].is_stop = True

        self.set_aperture(aperture_type="EPD", value=10)
        self.set_field_type(field_type="angle")
        self.add_field(y=0)
        self.add_field(y=5)
        self.add_field(y=10)

        self.add_wavelength(value=0.4861)  # F
        self.add_wavelength(value=0.5876, is_primary=True)  # d
        self.add_wavelength(value=0.6563)  # C

lens_system = CookeTripletStartPoint()

# Display initial lens
fig_initial = lens_system.draw(title="Initial Cooke Triplet")
display(fig_initial)
plt.close(fig_initial) # Close to prevent double display in some environments
lens_system.info()

## 2. Define the Optimization Problem

We'll set up operands to minimize RMS spot size and maintain a target focal length. Variables will include radii, thicknesses, and the materials of the lenses.

In [ ]:
problem = optimization.OptimizationProblem(optic=lens_system)
target_focal_length = 50.0

# Operand: Effective Focal Length (EFL)
problem.add_operand(
    operand_type="effective_focal_length",
    target=target_focal_length,
    weight=1.0
)

# Operands: RMS Spot Size for each field
for field_coord in lens_system.fields.get_field_coords():
    problem.add_operand(
        operand_type="rms_spot_size",
        target=0.0,
        weight=10.0,
        input_data={
            "optic": lens_system,
            "surface_number": lens_system.surfaces.last_surface_index,
            "Hx": field_coord[0],
            "Hy": field_coord[1],
            "num_rays": 16, # Rays on one axis for 'uniform' distribution
            "wavelength": lens_system.wavelengths.primary_wavelength.value,
            "distribution": "uniform"
        }
    )

# Variables: Radii
problem.add_variable(lens_system, "radius", surface_number=1, min_val=10, max_val=100)
problem.add_variable(lens_system, "radius", surface_number=2, min_val=-200, max_val=-20)
problem.add_variable(lens_system, "radius", surface_number=3, min_val=-100, max_val=-10)
problem.add_variable(lens_system, "radius", surface_number=4, min_val=10, max_val=100)
problem.add_variable(lens_system, "radius", surface_number=5, min_val=20, max_val=200)
problem.add_variable(lens_system, "radius", surface_number=6, min_val=-100, max_val=-10)

# Variables: Thicknesses (air and glass)
problem.add_variable(lens_system, "thickness", surface_number=1, min_val=1, max_val=5) # Lens 1 thickness
problem.add_variable(lens_system, "thickness", surface_number=2, min_val=1, max_val=15) # Air space 1
problem.add_variable(lens_system, "thickness", surface_number=3, min_val=1, max_val=5) # Lens 2 thickness
problem.add_variable(lens_system, "thickness", surface_number=4, min_val=1, max_val=15) # Air space 2
problem.add_variable(lens_system, "thickness", surface_number=5, min_val=1, max_val=5) # Lens 3 thickness
problem.add_variable(lens_system, "thickness", surface_number=6, min_val=30, max_val=60) # Back focal length (approx)

# Variables: Materials (using Glass Expert)
available_glasses = glasses_selection(min_nd=1.45, max_nd=1.95, min_vd=20, max_vd=70, catalogs=["schott", "ohara_common"])
problem.add_variable(lens_system, "material", surface_number=1, glass_selection=available_glasses)
problem.add_variable(lens_system, "material", surface_number=3, glass_selection=available_glasses)
problem.add_variable(lens_system, "material", surface_number=5, glass_selection=available_glasses)

problem.info()

## 3. Run the Glass Expert Optimization

In [ ]:
optimizer = optimization.GlassExpert(problem)

optic_history = []
error_history = []

# Create display handles for updating plots live
lens_disp_handle = display(display_id=True)
error_disp_handle = display(display_id=True)

def optimization_callback(*args):
    # Store current state for plotting history
    optic_history.append(lens_system.copy())
    error_history.append(optimizer.problem.sum_squared())
    
    # Update lens plot (optional, can slow down optimization)
    # if len(optic_history) % 5 == 0: # Update every 5 iterations
    #     fig_lens = plot_optic_history_matplotlib(optic_history, iteration=-1, title_comment="Optimizing...")
    #     lens_disp_handle.update(fig_lens)
    #     plt.close(fig_lens)
    
    # Update error plot
    fig_error = plot_error_history_matplotlib(error_history)
    error_disp_handle.update(fig_error)
    plt.close(fig_error)
    
    # Clear previous lens plot output to save space if not updating it every time
    # lens_disp_handle.update(None) 
    clear_output(wait=True) # Clears all previous outputs, use with caution or refine display updates

print(f"Initial Merit Function: {problem.initial_value:.4f}")

result = optimizer.run(
    num_neighbours=5,  # Reduced for faster example
    maxiter=50,        # Reduced for faster example
    tol=1e-4,
    callback=optimization_callback,
    verbose=True,
    plot_glass_map=False # Disable glass map plotting for gallery example
)

# Final plots after optimization
clear_output(wait=True)
fig_final_lens = lens_system.draw(title=f"Optimized Cooke Triplet (Merit: {result.fun:.4f})")
display(fig_final_lens)
plt.close(fig_final_lens)

if error_history:
    fig_final_error = plot_error_history_matplotlib(error_history, title="Merit Function Evolution")
    display(fig_final_error)
    plt.close(fig_final_error)

lens_system.info()
print(f"\nFinal glasses: {[v.value for v in problem.variables if v.type == 'material']}")

## 4. Analyze Results

The plots above show the lens system before and after optimization, and the evolution of the merit function. The `GlassExpert` helps in finding a good combination of real glasses from the provided catalog, alongside optimizing the continuous parameters.